Wrangling and Cleaning WeRateDogs Twitter Archive
=================================================

# Introduction

This dataset comes from an archive of tweets from the Twitter user @dog_rates, also known as WeRateDogs. This account rates people's dogs along with a humorous comment about the dog. The dog then gets a rating that almost always has a denominator of 10. The numerators on the other hand, almost always go beyond 10, [because they're good dogs Brent](https://knowyourmeme.com/memes/theyre-good-dogs-brent). The account has over 7 million followers as of 2018. In this notebook, I will wrangle, clean, and analyze the twitter archive and other supporting data.

### The Process

The data for this project requires gathering from a few different sources. The twitter archive, `df_twitter`, comes from a file given to Udacity by WeRateDogs. The other file, `image_predictions` comes from Udacity's servers and will be downloaded by utilizing the `requests` library. All other data will be gathered by accessing the Twitter API and extracting the JSON of each tweet. I will then use the extracted JSON to add extra data to `df_twitter`. I also use the `BeautifulSoup` library to extract users that were tagged in the photos of the tweet. Once I gather all the data I need, I will clean and visualize it.

### Twitter Archive

The archive was downloaded using [this link](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv). In the table there are a total of 17 columns. Most of these columns are described on the [twitter developer web page](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html) on tweet objects. 

### Image Predictions

This file was downloaded using [this link](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv) and the `requests` library to save the file for analysis. The column descriptions are as follows:

* tweet_id is the last part of the tweet URL after "status/" → https://twitter.com/dog_rates/status/889531135344209921
* p1 is the algorithm's #1 prediction for the image in the tweet → golden retriever
* p1_conf is how confident the algorithm is in its #1 prediction → 95%
* p1_dog is whether or not the #1 prediction is a breed of dog → TRUE
* p2 is the algorithm's second most likely prediction → Labrador retriever
* p2_conf is how confident the algorithm is in its #2 prediction → 1%
* p2_dog is whether or not the #2 prediction is a breed of dog → TRUE
* etc.

# Wrangling

In [3]:
import requests
import pandas as pd
import tweepy
import time
import json
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import os

In [102]:
# get image-predictions.tsv with requests library
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

In [103]:
with open('image-predictions.tsv', 'wb') as file:
    file.write(r.content)

In [4]:
image_predictions = pd.read_csv('image-predictions.tsv', sep = '\t')

In [232]:
image_predictions.head()

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True  
3  0.360687    True   miniature_pinscher  0.222752    True  
4  0.243682    True             Doberman  0.154629    True

In [106]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


The `image_predictions` file is pretty tidy. From only looking at the first five rows, I can already see that I will probably only need the `tweet_id`, `jpg_url`, `p1`, and `p1_conf` columns of the rows that contain `True` in `p1_dog`. The other columns have a very low confidence percentage and will not be necessary to add to the data. Next lets look at the twitter archive.

In [5]:
df_twitter = pd.read_csv('twitter-archive-enhanced.csv')

In [108]:
df_twitter.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3  This is Darla. She commenced a snooze mid meal...                  NaN   
4  This is Franklin. He would like you to stop ca...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   
2  https://twitter.com/dog_rates/status/891815181...                12   
3  https://twitter.com/dog_rates/status/891689557...                13   
4  https://twitter.com/dog_rates/status/891327558...                12   

   rating_denominator      name doggo floofer pupper puppo  
0                  10   Phineas  None    None   None  None  
1                  10     Tilly  None    None   None  None  
2                  10    Archie  None    None   None  None  
3                  10     Darla  None    None   None  None  
4                  10  Franklin  None    None   None  None

In [109]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

`df_twitter` is pretty messy. Just by taking a glance at the first five rows, we can already see that there are numerous tidy and quality issues with the data. We have NaN values, None values, the strings in `source` need to be cleaned up, ect. 

Next, I will use the twitter API to access the JSON of each tweet. I will use each ID from the `tweet_id` column in `df_twitter`, save the JSON to a text file, then use that text file to find more data on each tweet.

In [111]:
# oauth authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_url)

In [112]:
try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')

In [113]:
# access the api using the tweepy library
api = tweepy.API(auth, 
                 parser = tweepy.parsers.JSONParser(),
                 wait_on_rate_limit = True,
                 wait_on_rate_limit_notify = True)

Below, I wrote a script that loops through the `tweet_id` column and uses each ID to access the tweet JSON. I then store all of the json in a list, which then gets outputed to a text file at the end of the script. While the loops is running, it catches errors and stores the ID in a list called `tweet_errors`. Also, the script will print out any ID that his has errors with. It's nice to be able to track what your script is doing when it runs for such a long time. When you access the twitter API too much, you reach your rate limit. Once this happens, the script gets "put to sleep" for 15 to 30 minutes. This was one of the biggest challenges of this project. Since it takes roughly an hour for this script to run, it needed to be tested on a much smaller scale so there are zero errors when used on the dataset.

While writing this script, I stumbled upon a very useful technique for gathering data. This method comes from the `collections` library. I imported a method called `defaultdict`. Essentially it makes a basic python dictionary, but with something a little extra. If the key in the dictionary is not present, it will put a new key with it's value in the dictionary. Once you need to add another value to the same key, it will add the value to the same key as a list. So no matter what, you will create a dictionary that you can later turn into a dataframe with a few lines of code.

Once the JSON is dumped and the dictionary is turned into a dataframe, the dataframe is then turned into a csv file for later cleaning and analysis.

In [19]:
# start timer
start_time = time.time()
# list of tweet json
tweet_json = []
# dictionary of extracted tweet data
tweet_dict = defaultdict(list)
# errors (if any) of tweet extraction
tweet_errors = []

for tweet_id in df_twitter.tweet_id:
    try:
        # get tweet
        tweet = api.get_status(tweet_id, tweet_mode = 'extended')
        # append json to tweet_json list
        tweet_json.append(tweet)
        # extract tweet data
        tweet_dict['retweet_count'].append(tweet['retweet_count'])
        tweet_dict['favorite_count'].append(tweet['favorite_count'])
        tweet_dict['tweet_id'].append(tweet['id'])   
    except:
        # if tweet is not extracted, append id to tweet_errors
        tweet_errors.append(tweet_id)
        # print id with error
        print('error in extracting' + ' ' + str(tweet_id))
# dump json into txt file
with open('tweet_json.txt', 'w', encoding = 'utf8') as tweettxtfile:
    json.dump(tweet_json, tweettxtfile, sort_keys = True, indent = 4, ensure_ascii = False)
# dict to dataframe
df_twitter_extract = pd.DataFrame.from_dict(tweet_dict)
# dataframe to csv
df_twitter_extract.to_csv('df_twitter_extract.csv', encoding='utf-8')
print('Script took %s seconds to run' % (time.time() - start_time))
error_length = len(tweet_errors)
print('There were %s tweet errors when the script was ran' % error_length)

error in extracting 888202515573088257
error in extracting 873697596434513921
error in extracting 869988702071779329
error in extracting 866816280283807744
error in extracting 861769973181624320
error in extracting 845459076796616705
error in extracting 842892208864923648
error in extracting 837012587749474308
error in extracting 827228250799742977
error in extracting 802247111496568832
error in extracting 775096608509886464
error in extracting 771004394259247104
error in extracting 770743923962707968


Rate limit reached. Sleeping for: 325


error in extracting 754011816964026368


Rate limit reached. Sleeping for: 680


Script took 1618.222422361374 seconds to run
There were 14 tweet errors when the script was ran


Surprisingly, there were only 14 errors when the script was ran. After checking the `tweet_json.txt`, there are a total of 461,441 lines of JSON. For the file I used the `sort_keys`, `indent`, and `ensure_ascii` to make a clean and readable json file. This helped later on when I needed to view a twitter object in the JSON file to figure out what code I needed to access certain elements.

For the next part, I wanted to add some extra data to the project. I wanted to extract the JSON of the user's who were getting tagged in certain tweets on WeRateDogs. Using the ID's from `twitter_df`, I accessed the url's of the tweets with the `BeautifulSoup` library. Then, I saved the html of each url in a folder for later use. I named each file by splitting the string of the url and adding the ID to the file name.

In [ ]:
start_time = time.time()

html_tweet_dict = defaultdict(list)

html_tweet_errors = []
for tweet_id in df_twitter.tweet_id:
    try:
        url = api.get_status(tweet_id)['entities']['urls'][0]['expanded_url']
        r = requests.get(url)
        html_tweet_dict['link'].append(url)
        html_tweet_dict['tweet_id'].append(tweet_id)
        # access html with BeautifulSoup
        soup = BeautifulSoup(r.text, 'html.parser').prettify()
        # save html of tweet to tweet_html folder
        with open('tweet_html/html_' + url.split('/')[-1] + '.html', 'w', encoding = 'utf-8') as file:
            file.write(str(soup))
    except:
        html_tweet_errors.append(tweet_id)
        print('error in obtaining url from tweet ID %s' % tweet_id)

# turn dict into dataframe
df_html_extract = pd.DataFrame.from_dict(html_tweet_dict)
# dataframe to csv
df_html_extract.to_csv('df_html_extract.csv', encoding='utf-8')

print('Script took %s seconds to run' % (time.time() - start_time))
html_error_length = len(html_tweet_errors)
print('There were %s user tweet errors when the script was ran' % html_error_length)

In [45]:
len(html_tweet_errors)

1973

After running this script, there were a total of 1973 errors. The cause of the errors was the `url` variable in the loop. Due to some tweets being a bit old, the JSON of those tweets were different than others. So to narrow down the number of errors. I searched for another way to grab the url from the JSON and ran the script again.

In [ ]:
start_time = time.time()

html_tweet_errors_2 = []

# loop through html_tweet_errors
for tweet_id in html_tweet_errors:
    try:
        # changed strings to access correct link
        url = api.get_status(tweet_id)['entities']['media'][0]['expanded_url']
        r = requests.get(url)
        html_tweet_dict['link'].append(url)
        html_tweet_dict['tweet_id'].append(tweet_id)
        soup = BeautifulSoup(r.text, 'html.parser').prettify()
        # save html of tweet to tweet_html folder (changed index to -3)
        with open('tweet_html/html_' + url.split('/')[-3] + '.html', 'w', encoding = 'utf-8') as file:
            file.write(str(soup))
    except:
        html_tweet_errors_2.append(tweet_id)
        print('error in obtaining url from tweet ID %s' % tweet_id)

df_html_extract_2 = pd.DataFrame.from_dict(html_tweet_dict)
# dataframe to csv
df_html_extract_2.to_csv('df_html_extract_.csv', encoding='utf-8')

print('Script took %s seconds to run' % (time.time() - start_time))
html_error_length_2 = len(html_tweet_errors_2)
print('There were %s user tweet errors when the script was ran' % html_error_length_2)

In [73]:
len(html_tweet_errors_2)

153

I narrowed it down to 153 errors. AFter some digging, most of the other 153 ID's url's didn't exist on twitter anymore, so I decided to move on.

With those two scripts ran, I now have the HTML for most of the tweets. I also have a CSV file called `df_html_extract` that has the ID and the url of that ID. This will be used later to add missing data to the `df_twitter` dataframe.

Next, I will use the HTML files extracted earlier and find the user ID of the profile tagged in the original tweet. After inspecting the link of the tagged user on twitter, I discovered that there was a `div` with a class called `media-tagging-block` that had a number in `data-user-id`. With this ID number, we can use `api.get_user` to extract the json just as we did with the original tweets.

So for the script I wrote below, it will loop through the html files, find the user ID number, extract the JSON of the user, and collect data from it. After all of the data is collected it puts it in a csv file called `df_user_extract`.

In [ ]:
# list of user tweet json
user_tweet_json = []
# tweet data of users tagged in original tweet
user_tweet_dict = defaultdict(list)
# list of errors
user_tweet_error = []
folder = 'tweet_html'
for tweet_html in os.listdir(folder):
    try:
        with open(os.path.join(folder, tweet_html), encoding = 'utf-8') as file:
            soup = BeautifulSoup(file, 'html.parser')
            # find user ID of profile tagged in original tweet
            user_id = soup.find('div', 'media-tagging-block')('a')[0].get('data-user-id')
            # get profile JSON using user ID
            user_json = api.get_user(user_id)
            # put json in user_tweet_json
            user_tweet_json.append(user_json)
            # extract user profile data
            user_tweet_dict['created_at'].append(user_json['created_at'])
            user_tweet_dict['friends_count'].append(user_json['friends_count'])
            user_tweet_dict['user_id'].append(user_json['id_str'])
            user_tweet_dict['language'].append(user_json['lang'])
            user_tweet_dict['location'].append(user_json['location'])
            user_tweet_dict['display_name'].append(user_json['name'])
            user_tweet_dict['account_name'].append(user_json['screen_name'])
    except:
        user_tweet_error.append(file)
        print('error in obtaining tagged user ID data from %s' % file)


In [158]:
with open('user_tweet_json.txt', 'w', encoding = 'utf-8') as usertxtfile:
    json.dump(user_tweet_json, usertxtfile, sort_keys = True, indent = 4, ensure_ascii = False)
df_user_extract = pd.DataFrame.from_dict(user_tweet_dict)
# dataframe to csv
df_user_extract.to_csv('df_user_extract.csv', encoding='utf-8')

In [159]:
len(user_tweet_error)

1412




With this script I was able to extract user data for 718 individuals. There were 1412 errors mostly because not all tweets had a tagged user in it. The data I was able to collect included the date in which the user joined twitter, friends count, user ID, language spoken, location of user, display name, and account name.

In summary, I collected the twitter archive of WeRateDogs, the HTMl of each tweet link, tagged user data, image predictions from Udacity's servers, and JSON for tagged users and for each tweet in the archive. The next step will be to clean and organize the data, so we have good quality and tidiness for analyzation. 

# Cleaning

To start out, lets take a look at each csv file we gathered to see what data we have to work with.

In [6]:
# make a copy of df_twitter to alter it later
df_twitter_copy = df_twitter.copy()
df_twitter_copy.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3  This is Darla. She commenced a snooze mid meal...                  NaN   
4  This is Franklin. He would like you to stop ca...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   
2  https://twitter.com/dog_rates/status/891815181...                12   
3  https://twitter.com/dog_rates/status/891689557...                13   
4  https://twitter.com/dog_rates/status/891327558...                12   

   rating_denominator      name doggo floofer pupper puppo  
0                  10   Phineas  None    None   None  None  
1                  10     Tilly  None    None   None  None  
2                  10    Archie  None    None   None  None  
3                  10     Darla  None    None   None  None  
4                  10  Franklin  None    None   None  None

In [7]:
image_predictions = pd.read_csv('image-predictions.tsv', sep = '\t')
image_predictions.head()

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True  
3  0.360687    True   miniature_pinscher  0.222752    True  
4  0.243682    True             Doberman  0.154629    True

In [8]:
df_html_extract = pd.read_csv('df_html_extract.csv')
df_html_extract.head()

Unnamed: 0                                               link  \
0           0  https://twitter.com/i/web/status/8921774213063...   
1           1  https://twitter.com/i/web/status/8918151813780...   
2           2  https://twitter.com/i/web/status/8913275589266...   
3           3  https://twitter.com/i/web/status/8910879508758...   
4           4  https://twitter.com/i/web/status/8909719131739...   

             tweet_id  
0  892177421306343426  
1  891815181378084864  
2  891327558926688256  
3  891087950875897856  
4  890971913173991426

In [9]:
# extra data extracted for twitter archive
df_twitter_extract = pd.read_csv('df_twitter_extract.csv')
df_twitter_extract.head()

Unnamed: 0  favorite_count  retweet_count            tweet_id
0           0           38489           8472  892420643555336193
1           1           32964           6237  892177421306343426
2           2           24829           4133  891815181378084864
3           3           41857           8597  891689557279858688
4           4           40017           9327  891327558926688256

In [10]:
df_user_extract = pd.read_csv('df_user_extract.csv')
df_user_extract.head()

Unnamed: 0     account_name                      created_at  \
0           0        dog_rates  Sun Nov 15 21:41:29 +0000 2015   
1           1          tta_kay  Sat Jul 05 17:06:22 +0000 2014   
2           2  Adeline_Garrett  Fri Apr 26 16:34:47 +0000 2013   
3           3          gwatsky  Sun Jan 10 03:38:19 +0000 2010   
4           4     SmilingYoshi  Sun Jun 12 20:38:49 +0000 2011   

         display_name  friends_count language                        location  \
0         WeRateDogs™             10       en  ⇩ merch ⇩         DM YOUR DOGS   
1                 kay            479       en                             NaN   
2  outofline carrot 🦊            415       en                         912•LDN   
3              W▵TSKY            865       en                            yeah   
4        SmilingYoshi           1412       en                           Here.   

      user_id  
0  4196983835  
1  2676442024  
2  1382344291  
3   103467620  
4   316025613

The only data frame that I will be changing and adding data too is `df_twitter`. Mainly because all of my other tables (except `df_user_extract`) have the `tweet_id`'s in each. One method you will see throughout this cleaning process is `map`. Conveniently, I can use the tweet ID's from my extracted data and map it to the twitter archive data. That way, we have the data next to the associated tweet ID.

After doing a visual analysis on the `df_twitter` table, I listed some quality and tidiness issues that need to be fixed:

### Quality Issues
* Change `tweet_id` column to a `string`
* Change `timestamp` to `datetime`
* Extract the text in each row for the `source column` with beautiful soup
* Change `doggo`, `floofer`, `pupper`, and `puppo` to boolean
* drop `expanded_urls` column, extract urls with tweet id's, then map urls to `df_twitter`
* Extract `retweeted` element from each object in `tweet_json` and map column to `df_twitter`
* Create a column that divides the numerator to the denominator to have a common rating system
* Fix any outliers in the `name` column, for example, "a" as a name

### Tidiness Issues
* remove the following rows: `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`
* after above step, remove rows that have missing data

### Misc.
* Extract image link, first confidence level, and breed of dogs that are `True` breeds from `image_predictions`

First I will start off by mapping the extra tweet archive data to `df_twitter`.

In [11]:
# map df_twitter_extract to df_twitter
df_twitter_copy['retweet_count'] = df_twitter_copy['tweet_id'].map(df_twitter_extract.set_index('tweet_id')['retweet_count'])
df_twitter_copy['favorite_count'] = df_twitter_copy['tweet_id'].map(df_twitter_extract.set_index('tweet_id')['favorite_count'])

Next, I extracted the `retweeted` status of each tweet and added it to the `df_twitter` table.

In [12]:
retweet_status_dict = defaultdict(list)
json_file = 'tweet_json.txt'
with open(json_file, encoding = 'utf-8') as f:
    data = json.load(f)
    for element in data:
        retweet_status_dict['retweeted'].append(element['retweeted'])
        retweet_status_dict['tweet_id'].append(element['id'])
    print('done!')
df_retweet_status = pd.DataFrame.from_dict(retweet_status_dict)

done!


In [13]:
df_retweet_status.head()

retweeted            tweet_id
0      False  892420643555336193
1      False  892177421306343426
2      False  891815181378084864
3      False  891689557279858688
4      False  891327558926688256

In [14]:
# add retweeted column to df_twitter
df_twitter_copy['retweeted'] = df_twitter_copy['tweet_id'].map(df_retweet_status.set_index('tweet_id')['retweeted'])

In [15]:
df_twitter_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 20 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
retweet_count                 23

Looking at the structure of `df_twitter_copy`, we see that there are a great number of missing values, and data types that should not be there. I decided that the columns `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, and `retweeted_status_timestamp` were not needed.

In [16]:
# drop columns
df_twitter_copy = df_twitter_copy.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis = 1)

In the column `expanded_urls`, there were a great number of values that were "None", to fix this, I used the urls that I extracted earlier in the `df_html_extract` table, and mapped it to `df_twitter`

In [17]:
# drop original expanded_urls column
df_twitter_copy = df_twitter_copy.drop('expanded_urls', axis = 1)

In [18]:
# map urls to df_twitter
df_twitter_copy['expanded_urls'] = df_twitter_copy['tweet_id'].map(df_html_extract.drop_duplicates('tweet_id').set_index('tweet_id')['link'])

In the next steps, I will turn the `tweet_id` column in to a string type, and `timestamp` into DateTime type. Since the ID numbers do not need to be calculated in any mathematical way and shouldn't be changed, they should be strings. The DateTime type for the `timestamp` column will be usefull for visualization when I need to put elements in order by time.

In [19]:
# turn tweet_id into string type
df_twitter_copy['tweet_id'] = df_twitter_copy['tweet_id'].astype('str')

In [20]:
# turn timestamp into DateTime
df_twitter_copy['timestamp'] = pd.to_datetime(df_twitter_copy['timestamp'])

For this next section I used BeautifulSoup to extract the content in each html tag for the `source` column. BeauifulSoup was the easiest method to use for parsing the html string. Once I was able to extract the content of each tag, I then used the `replace` method to change the source types into simple strings such as iphone, vine, web, and tweetdeck.

In [21]:
# function for accessing tag content
def source_text(html):
    source = BeautifulSoup(html, 'lxml')
    return source.a.string

# change each cell to tag content with lambda function
df_twitter_copy['source_text'] = df_twitter_copy.apply(lambda row: source_text(row['source']), axis = 1)

In [22]:
# drop original source column
df_twitter_copy.drop('source', axis = 1, inplace = True)

In [23]:
# dictionary of string replacements
replacements = {
    'Twitter for iPhone': 'iphone',
    'Vine - Make a Scene': 'vine',
    'Twitter Web Client': 'web',
    'TweetDeck': 'tweetdeck'
}

# apply replacements
df_twitter_copy['source_text'].replace(replacements, inplace = True)

Next, I used the same method as above for replacing values in the `doggo`, `floofer`, `pupper`, and `puppo` columns. In the original `df_twitter` table, each column either had the label "None" or the given category of the dog, "doggo" for example. So for each value, if there was a category name, it was switched to `True`, and for each "None" value, it was switched to `False`. These boolean values will allow me to easily analyze each column, `df_twitter.doggo.mean()` for example.

In [24]:
# columns to be changed
cols = ['doggo', 'floofer', 'pupper', 'puppo']


replacements_dog = {
    'doggo': True,
    'floofer': True,
    'pupper': True,
    'puppo': True,
    'None': False
}

df_twitter_copy[cols] = df_twitter_copy[cols].replace(replacements_dog)

Finally, we need to make sure there are no missing values in the dataset. Since we have collected everything we will need, we will not need rows that have missing data, as we were not able to collect data for them

In [25]:
# drop any rows that have NaN values
df_twitter_copy = df_twitter_copy.dropna()

Since the scores in this data vary in scale (14/10 , 150/164, for example) the best way to have a consistent score in the entire data set is to divide the numerator by the denominator. This will allow us to easily make visualizations that show us average score, scores over time, highest score, ect.

In [26]:
# divide numerator column by denominator column
df_twitter_copy['score_rating'] = df_twitter_copy.rating_numerator / df_twitter_copy.rating_denominator

Using the same `replace` method, we can change certain outliers in the `name` column. "a" is clearly not a name.

In [27]:
replacements_name = {
    'a': 'None',
    'the': 'None',
    'an': 'None',
    'officially': 'None',
    'O': "O'Malley"
}

df_twitter_copy['name'].replace(replacements_name, inplace = True)

In the next steps, I will focus on joining the `image_predictions` data with `df_twitter`. I will narrow the image predictions table down to only rows where `p1_dog` are `True`. We only want strings in the column that are actual dogs that exist. Along with the name of the breed, I'll also include the JPEG file of the dog, and the confidence level in which the algorithm calculated. Since the image predictions table get's narrowed down to about 500 less datapoints then the twitter archive, I'll have to fill in the missing data with either a string "None", or a zero for integer columns.

In [28]:
# make a copy
image_predictions_copy = image_predictions.copy()

In [29]:
# drop all rows where p1_dog is False
image_predictions_copy.drop(image_predictions_copy[image_predictions_copy.p1_dog == False].index, inplace = True)

In [30]:
# drop all other unneeded rows
image_predictions_copy = image_predictions_copy.drop(['img_num', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], axis = 1)

In [31]:
# reset index 
image_predictions_copy = image_predictions_copy.reset_index(drop = True)

In [32]:
# make each tweet_id column identical for mapping
image_predictions_copy['tweet_id'] = image_predictions_copy['tweet_id'].astype(str)

In [33]:
# map data to the twitter archive
df_twitter_copy['jpg_url'] = df_twitter_copy['tweet_id'].map(image_predictions_copy.set_index('tweet_id')['jpg_url'])
df_twitter_copy['p1'] = df_twitter_copy['tweet_id'].map(image_predictions_copy.set_index('tweet_id')['p1'])
df_twitter_copy['p1_conf'] = df_twitter_copy['tweet_id'].map(image_predictions_copy.set_index('tweet_id')['p1_conf'])

In [34]:
# fill in NaN values
df_twitter_copy[['jpg_url', 'p1']] = df_twitter_copy[['jpg_url', 'p1']].fillna('None')
df_twitter_copy['p1_conf'] = df_twitter_copy['p1_conf'].fillna(0)

In [35]:
df_twitter_copy.head()

tweet_id           timestamp  \
0  892420643555336193 2017-08-01 16:23:56   
1  892177421306343426 2017-08-01 00:17:27   
2  891815181378084864 2017-07-31 00:18:03   
3  891689557279858688 2017-07-30 15:58:51   
4  891327558926688256 2017-07-29 16:00:24   

                                                text  rating_numerator  \
0  This is Phineas. He's a mystical boy. Only eve...                13   
1  This is Tilly. She's just checking pup on you....                13   
2  This is Archie. He is a rare Norwegian Pouncin...                12   
3  This is Darla. She commenced a snooze mid meal...                13   
4  This is Franklin. He would like you to stop ca...                12   

   rating_denominator      name  doggo  floofer  pupper  puppo  retweet_count  \
0                  10   Phineas  False    False   False  False         8472.0   
1                  10     Tilly  False    False   False  False         6237.0   
2                  10    Archie  False    False   False  False         4133.0   
3                  10     Darla  False    False   False  False         8597.0   
4                  10  Franklin  False    False   False  False         9327.0   

   favorite_count retweeted  \
0         38489.0     False   
1         32964.0     False   
2         24829.0     False   
3         41857.0     False   
4         40017.0     False   

                                       expanded_urls source_text  \
0  https://twitter.com/dog_rates/status/892420643...      iphone   
1  https://twitter.com/i/web/status/8921774213063...      iphone   
2  https://twitter.com/i/web/status/8918151813780...      iphone   
3  https://twitter.com/dog_rates/status/891689557...      iphone   
4  https://twitter.com/i/web/status/8913275589266...      iphone   

   score_rating                                          jpg_url         p1  \
0           1.3                                             None       None   
1           1.3  https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg  Chihuahua   
2           1.2  https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg  Chihuahua   
3           1.3                                             None       None   
4           1.2  https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg     basset   

    p1_conf  
0  0.000000  
1  0.323581  
2  0.716012  
3  0.000000  
4  0.555712

In [36]:
df_twitter_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2206 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id              2206 non-null object
timestamp             2206 non-null datetime64[ns]
text                  2206 non-null object
rating_numerator      2206 non-null int64
rating_denominator    2206 non-null int64
name                  2206 non-null object
doggo                 2206 non-null bool
floofer               2206 non-null bool
pupper                2206 non-null bool
puppo                 2206 non-null bool
retweet_count         2206 non-null float64
favorite_count        2206 non-null float64
retweeted             2206 non-null object
expanded_urls         2206 non-null object
source_text           2206 non-null object
score_rating          2206 non-null float64
jpg_url               2206 non-null object
p1                    2206 non-null object
p1_conf               2206 non-null float64
dtypes: bool(4), datetime64[ns](1), float64(4), int64(2), object(8)
memory

Lastly, I will do some quick cleaning on the `df_user_extract` dataframe. Let's see what we need to do.

In [37]:
df_user_extract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718 entries, 0 to 717
Data columns (total 8 columns):
Unnamed: 0       718 non-null int64
account_name     718 non-null object
created_at       718 non-null object
display_name     718 non-null object
friends_count    718 non-null int64
language         718 non-null object
location         457 non-null object
user_id          718 non-null int64
dtypes: int64(3), object(5)
memory usage: 45.0+ KB


### Quality
* make user_id a string column type
* fill in NaN values for location with "not given"

### Tidiness
* remove unnamed column

In [38]:
df_user_extract['user_id'] = df_user_extract['user_id'].astype(str)
df_user_extract = df_user_extract.drop('Unnamed: 0', axis = 1)
df_user_extract['location'] = df_user_extract['location'].fillna("not given")

In [39]:
df_user_extract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718 entries, 0 to 717
Data columns (total 7 columns):
account_name     718 non-null object
created_at       718 non-null object
display_name     718 non-null object
friends_count    718 non-null int64
language         718 non-null object
location         718 non-null object
user_id          718 non-null object
dtypes: int64(1), object(6)
memory usage: 39.3+ KB


Now that we have two clean and tidy datasets to work with, lets export them each to a csv file.

In [40]:
df_twitter_copy.to_csv('twitter_archive_master.csv', index = False, encoding='utf-8')
df_user_extract.to_csv('user_extract_master.csv', index = False, encoding = 'utf-8')